In [1]:
import sys
import os

# Get current working directory (e.g., notebook location)
notebook_dir = os.getcwd()

# Construct the path to the correct directory
parent_dir = os.path.abspath(os.path.join(notebook_dir, '../src'))

# Add it to sys.path
sys.path.insert(0, parent_dir)

from functions.loading import load_csv
from functions.merging import merge_dataframes
from functions.imputation import (
    impute_with_mean,
    add_cyclical_features,
    add_lag_and_rolling_features,
)
from functions.remerge import remerge
from functions.train import train_model
from functions.test import test_model, get_min_from_csv
from functions.submission import create_submission
from functions.encoding import removal_nonnumeric_columns
from functions.split import split_data
from functions.predict import make_prediction

In [2]:
# read data
train_features = load_csv("../data/01_raw/dengue_features_train.csv")
train_labels = load_csv("../data/01_raw/dengue_labels_train.csv")
test_features = load_csv("../data/01_raw/dengue_features_test.csv")

df = merge_dataframes(
    train_features, train_labels, test_features, on=["city", "weekofyear", "year"]
)
df = add_cyclical_features(df)

df = add_lag_and_rolling_features(df)
df = impute_with_mean(df, "total_cases")

  city  year  weekofyear week_start_date  total_cases  month  dayofweek  \
0   sj  1990          18      1990-04-30          4.0      4          0   
1   sj  1990          19      1990-05-07          5.0      5          0   
2   sj  1990          20      1990-05-14          4.0      5          0   
3   sj  1990          21      1990-05-21          3.0      5          0   
4   sj  1990          22      1990-05-28          6.0      5          0   

   month_sin  weekofyear_sin  dayofweek_sin  ...  ndvi_ne_lag3  ndvi_ne_lag4  \
0   0.866025        0.845596            0.0  ...           NaN           NaN   
1   0.500000        0.776524            0.0  ...           NaN           NaN   
2   0.500000        0.696551            0.0  ...           NaN           NaN   
3   0.500000          0.6068            0.0  ...        0.1226           NaN   
4   0.500000        0.508531            0.0  ...        0.1699        0.1226   

   ndvi_ne_rollmean3  ndvi_ne_rollmean5  ndvi_nw_lag1  ndvi_nw_lag2 

/home/sascha/Documents/DSR/DengAI--Competition/src/functions/imputation.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_lag{lag}"] = df.groupby("city")[col].shift(lag)
/home/sascha/Documents/DSR/DengAI--Competition/src/functions/imputation.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_lag{lag}"] = df.groupby("city")[col].shift(lag)
/home/sascha/Documents/DSR/DengAI--Competition/src/functions/imputation.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of call

new feature: binning the temp and make it an dinary column for 2 stages: egg, larva & pupa, adult
temperature limits: 

- larva:  15-35 : good, 3 and 4 weeks lag
- adult: 10 - 39: good, 1 and 2 weeks lag

In [ ]:
df['good_larva_4'] = ((df['station_avg_temp_c_lag4'] >= 15) & (df['station_avg_temp_c_lag4'] <= 35)).astype(int)
df['good_larva_3'] = ((df['station_avg_temp_c_lag3'] >= 15) & (df['station_avg_temp_c_lag3'] <= 35)).astype(int)

df['good_larva_4-2'] = ((df['reanalysis_air_temp_k_lag4'] >= 15) & (df['reanalysis_air_temp_k_lag4'] <= 35)).astype(int)
df['good_larva_3-2'] = ((df['reanalysis_air_temp_k_lag3'] >= 15) & (df['reanalysis_air_temp_k_lag3'] <= 35)).astype(int)

df['good_adult_2'] = ((df['station_avg_temp_c_lag2'] >= 10) & (df['station_avg_temp_c_lag2'] <= 39)).astype(int)
df['good_adult_1'] = ((df['station_avg_temp_c_lag1'] >= 10) & (df['station_avg_temp_c_lag1'] <= 39)).astype(int)

df['good_adult_2'] = ((df['reanalysis_air_temp_k_lag2'] >= 10) & (df['reanalysis_air_temp_k_lag2'] <= 39)).astype(int)
df['good_adult_1'] = ((df['reanalysis_air_temp_k_lag1'] >= 10) & (df['reanalysis_air_temp_k_lag1'] <= 39)).astype(int)


/tmp/ipykernel_110394/1151437747.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['good_larva_4'] = ((df['station_avg_temp_c_lag4'] >= 15) & (df['station_avg_temp_c_lag4'] <= 35)).astype(int)
/tmp/ipykernel_110394/1151437747.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['good_larva_3'] = ((df['station_avg_temp_c_lag3'] >= 15) & (df['station_avg_temp_c_lag3'] <= 35)).astype(int)
/tmp/ipykernel_110394/1151437747.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

In [ ]:
df.reanalysis_air_temp_k_lag4
df = removal_nonnumeric_columns(df)
train_X, test_X, train_y, test_y, validation = split_data(df)
train_X, test_X, train_y, test_y, validation = split_data(df)

model = train_model(train_X, train_y)
best_MAE = 50

current_MAE = test_model(model, test_X, test_y)

if current_MAE >= best_MAE:
    print("Model did not perform better than previous models. Aborting.")
    print("Model did not perform better than previous models. Aborting.")

else:
    # CHECK IF MAE IS GOOD ENOUGH
    X = remerge(train_X, test_X)
    y = remerge(train_y, test_y)

    final_model = train_model(X, y)

    predictions = make_prediction(validation, final_model)

    create_submission(
        predictions, validation, "../data/03_submissions/naive_model_30_04_1314.csv"
    )

/home/sascha/Documents/DSR/DengAI--Competition/src/functions/encoding.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["city"] = df["city"].replace({"sj": 1, "iq": 2})


Mean absolute Error is 15.256674950082582
Submission file created at ../data/03_submissions/naive_model_30_04_1314.csv
